In [1]:
import sys
import soundcloud

In [2]:
from functools import partial
from utils import get_results, handle_http_errors

In [3]:
def getUserAttr(resource, attr):
#   if hasattr(resource, 'user'): return resource.user[attr]
    if hasattr(resource, attr): return getattr(resource, attr)
    return None

getUsername = partial(getUserAttr, attr='username')
getUserid = partial(getUserAttr, attr='id')

In [4]:
# A global artist graph used to iterate through the various algorithms.
# Each node is artist id, with edges weighted by activity between then.
#profileGraph = nx.MultiDiGraph()

client = soundcloud.Client(client_id='454aeaee30d3533d6d8f448556b50f23')

name = input("Enter a soundcloud artist to analyze: ")


Enter a soundcloud artist to analyze: Sybyr


In [5]:
# Artist of interest
search = client.get('/users/', q = name)[0]

In [6]:
search

In [7]:
search.__dict__

{'obj': {'avatar_url': 'https://i1.sndcdn.com/avatars-cIS3ELGYY7j2e0q9-OEySHA-large.jpg',
  'id': 13377746,
  'kind': 'user',
  'permalink_url': 'http://soundcloud.com/sybyr',
  'uri': 'https://api.soundcloud.com/users/13377746',
  'username': 'Sybyr',
  'permalink': 'sybyr',
  'last_modified': '2020/07/17 04:43:29 +0000',
  'first_name': 'Sybyrmusic.com',
  'last_name': '',
  'full_name': 'Sybyrmusic.com',
  'city': '',
  'description': 'bookings/email: sybyrcentral@gmail.com\nAnti-World: http://antiworld.co (@antiworldglobe) on all networks\n(Prod. by Landfill) at http://prodbylandfill.com',
  'country': 'United States',
  'track_count': 436,
  'public_favorites_count': 0,
  'followers_count': 51706,
  'followings_count': 22,
  'plan': 'Pro Unlimited',
  'myspace_name': None,
  'discogs_name': None,
  'website_title': 'click Here',
  'website': 'http://antiworld.lnk.to/SYBYR',
  'reposts_count': 99,
  'comments_count': 110,
  'online': False,
  'likes_count': 0,
  'playlist_count': 3

In [8]:
print("Artist interpreted as: %s" % getUsername(search))

Artist interpreted as: Sybyr


In [39]:
id2username_cache = {}

In [9]:
@handle_http_errors
def id2username(profile, kind='users'):
    global id2username_dict
    username = id2username_cache.get(profile, None)
    if username is not None: return username

    # username is none, we don't have it in cache
    result = client.get('/%s/%s' % (kind, str(profile)))
    if kind == 'comments':
        username = result.user['username']
    elif kind == 'tracks':
        username = result.title
    else:
        username = result.username
    # encode it correctly
    username = str(username.encode('utf-8'))
    id2username_cache[profile] = username
    return username

# Adding more functions

In [33]:
def getRelationships(profile, client, url): 
    return get_results(client, url)

@handle_http_errors
def getFollowings(profile):
    followings = get_results(client, '/users/{0:s}/followings/'.format(str(profile)))
    return followings

@handle_http_errors
def getFollowers(profile):
    followers = get_results(client, '/users/{0:s}/followers/'.format(str(profile)))
    return followers

@handle_http_errors
def getFavorites(profile):
    favorites = get_results(client, '/users/{0:s}/favorites/'.format(str(profile)))
    return favorites

@handle_http_errors
def getComments(profile):
    comments = get_results(client, '/users/{0:s}/comments/'.format(str(profile)))
    return comments

@handle_http_errors
def getTracks(profile):
    tracks = get_results(client, '/users/{0:s}/tracks/'.format(str(profile)))
    return tracks

In [34]:
# list of profiles to query
profiles_to_query = [search.id]

In [35]:
profiles_to_query

[13377746]

In [38]:
depth = 2
i = 0

# list of profiles we could not query
unavailable_profiles = []

In [46]:
for t in range(depth):

    print("Iteration " + str(t))

    profiles_to_query = list(set(profiles_to_query))

    for profile in profiles_to_query:
        username = id2username(profile)
        if username:
            print("\t", "Enqueueing: %s (%s)" % (username, profile))

            metFollowings = getFollowings(profile)
            print("met followings: " + ", ".join([getUsername(user) if isinstance(getUsername(user), str) else str(getUserid(user)) for user in metFollowings]))

Iteration 0
	 Enqueueing: b'Sybyr' (13377746)
met followings: @DevinWillDo, OWINILLSIN {Anti_World}, ERIC NORTH (@RI0T_ANGEL), ANTI-WORLD, MONĀE, neighborhood arion, B*BE, JULIEN ANDREAS, Raz Nein, lil xelly (@lilxelly), Landfill, Shark, RIOT_ANGEL, LIL MAI 2020, win32, Ghostie, PE$O EVANS, WIFIGAWD AKA UPT SOULJAH, DariBroko, Leon'sWOLF, CHACHI
Iteration 1
	 Enqueueing: b'Sybyr' (13377746)
met followings: @DevinWillDo, OWINILLSIN {Anti_World}, ERIC NORTH (@RI0T_ANGEL), ANTI-WORLD, MONĀE, neighborhood arion, B*BE, JULIEN ANDREAS, Raz Nein, lil xelly (@lilxelly), Landfill, Shark, RIOT_ANGEL, LIL MAI 2020, win32, Ghostie, PE$O EVANS, WIFIGAWD AKA UPT SOULJAH, DariBroko, Leon'sWOLF, CHACHI


In [21]:
for profile in profiles_to_query:
    
    basic_info = []
    
    followings = getFollowings(profile)
    basic_info.append(followings)
    
    followers = getFollowers(profile)
    basic_info.append(followers)
    
    favorites = getFavorites(profile)
    basic_info.append(favorites)
    
    comments = getComments(profile)
    basic_info.append(comments)
    
    tracks = getTracks(profile)
    basic_info.append(tracks)
    print(basic_info)

[<generator object get_results.<locals>.yield_results at 0x7ff6099094d0>, <generator object get_results.<locals>.yield_results at 0x7ff609909550>, <generator object get_results.<locals>.yield_results at 0x7ff6099091d0>, <generator object get_results.<locals>.yield_results at 0x7ff608efac50>, <generator object get_results.<locals>.yield_results at 0x7ff6087999d0>]


Tested the five functions above ^ to successfully generate a get_results object